# Load Libraries and Set Settings

In [1]:
# Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2
from io import BytesIO
from tqdm import tqdm
import time
import csv
import sys
from random import randint

# Other Settings
pd.set_option('display.max.colwidth', None) # max display width

# Load Data

In [18]:
# Load data from CSV file into a DataFrame
article_link_directory = pd.read_csv('article_link_directory.csv')

### Subset the data for quick-time tests

In [19]:
practice_article_link_directory = article_link_directory.head(50)

In [20]:
practice_article_link_directory.head(1)

,Content Links,Publishing Info,Collected Links,Outside PDF Text,Full Collected Links
0,https://www.regeringen.se/remisser/2023/11/inbjudan-till-larosaten-myndigheter-och-organisationer-att-inkomma-med-synpunkter-till-regeringens-forskningspolitik/,Publicerad02 november 2023· Uppdaterad10 november 2023·RemissfrånUtbildningsdepartementet,/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf,NO CONTENT,https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf


# Collect Text Data from PDF Links

### Collect Data (New Way)

In [21]:
# No content alert
no_content_alert = "NO CONTENT"

In [22]:
# Specify the path to the CSV file
csv_file_path = "pdf_data.csv"

# Open the CSV file in write mode
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header row to the CSV file
    csv_writer.writerow(['PDF URL', 'Inside PDF Text'])

    for i, pdf_url in enumerate(tqdm(practice_article_link_directory['Full Collected Links'], desc="Step 2: Collecting PDF Text", unit="link")):
        # Check if the URL contains the NO CONTENT alert
        if "NO CONTENT" in pdf_url:
            # Write the data to the CSV file
            csv_writer.writerow([pdf_url, "NO CONTENT"])
            continue

        else:
            try:
                # Send an HTTP GET request to the PDF URL
                response = requests.get(pdf_url)

                # Check if the request was successful (status code 200)
                if response.status_code == 200:
                    # Wrap the response content in a BytesIO object
                    pdf_bytes = BytesIO(response.content)

                    # Create a PDF reader object
                    pdf_reader = PyPDF2.PdfReader(pdf_bytes)

                    # Initialize an empty string to store the text data
                    text_data = ""

                    # Use len(reader.pages) to determine the number of pages
                    for page_num in range(len(pdf_reader.pages)):
                        page = pdf_reader.pages[page_num]
                        text_data += page.extract_text()

                    # Write the data to the CSV file
                    csv_writer.writerow([pdf_url, text_data])
                else:
                    print(f"Failed to fetch PDF URL: {pdf_url}, Status code: {response.status_code}")
                    # Write the data to the CSV file
                    csv_writer.writerow([pdf_url, "NO CONTENT"])

            except Exception as e:
                print(f"An error occurred while processing PDF URL: {pdf_url}, Error: {str(e)}")
                # Write the data to the CSV file
                csv_writer.writerow([pdf_url, "NO CONTENT"])

        # Introduce a delay time before the next request
        time.sleep(3)  # Adjust the delay time as needed

        # Add a break statement if the loop index is equal to the expected number of links minus 1
        if i == len(practice_article_link_directory['Full Collected Links']) - 1:
            break



Step 2: Collecting PDF Text:  98%|█████████▊| 49/50 [04:55<00:06,  6.02s/link]


In [26]:
test = pd.read_csv('pdf_data.csv')

In [25]:
len(test)

50

In [24]:
test.head(3)

PDF URL  \
0  https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   
1     https://www.regeringen.se/contentassets/f0bce7ab7cb94f7ca766b01dc1399307/uppdrag-om-informationsinsatser-for-minskad-energianvandning-i-statlig-verksamhet.pdf   
2                                                              https://www.regeringen.se/contentassets/18dbd317274c488997ede753f6df2670/remissmissiv-sou-2023_24.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

### Collect data (old way)

In [43]:
# COLLECT PDF TEXT CONTENT

# Initialize an empty list to collect text data from PDFs
pdf_text_data = []

for pdf_url in tqdm(practice_article_link_directory['Full Collected Links'], desc = "Step 2: Collecting PDF Text", unit = "link"):
    
    # Check if the URL contains the NO CONTENT alert
    if "NO CONTENT" in pdf_url:
        # If yes, append "NO CONTENT" to pdf_text_data and continue to the next URL
        pdf_text_data.append("NO CONTENT")

    else:
        try:
            # Send an HTTP GET request to the PDF URL
            response = requests.get(pdf_url)

            # Check if the request was successful (status code 200)
            if response.status_code == 200:
                # Wrap the response content in a BytesIO object
                pdf_bytes = BytesIO(response.content)

                # Create a PDF reader object
                pdf_reader = PyPDF2.PdfReader(pdf_bytes)

                # Initialize an empty string to store the text data
                text_data = ""

                # Use len(reader.pages) to determine the number of pages
                for page_num in range(len(pdf_reader.pages)):
                    page = pdf_reader.pages[page_num]
                    text_data += page.extract_text()

                # Append the extracted text to the pdf_text_data list
                pdf_text_data.append(text_data)
            else:
                print(f"Failed to fetch PDF URL: {pdf_url}, Status code: {response.status_code}")
                pdf_text_data.append("NO CONTENT")

        except Exception as e:
            print(f"An error occurred while processing PDF URL: {pdf_url}, Error: {str(e)}")
            pdf_text_data.append("NO CONTENT")

    # Introduce a random delay time before the next request
    time.sleep(3)  # Adjust the delay time as needed


Step 2: Collecting PDF Text: 100%|██████████| 10/10 [00:55<00:00,  5.55s/link]


In [44]:
# Create a new column "Inside PDF Text" in article_link_directory and assign pdf_text_data to it
practice_article_link_directory['Inside PDF Text'] = pdf_text_data

C:\Users\ryanh\AppData\Local\Temp\ipykernel_28180\3828830412.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  practice_article_link_directory['Inside PDF Text'] = pdf_text_data


In [45]:
practice_article_link_directory.head(1)

Content Links  \
0  https://www.regeringen.se/remisser/2023/11/inbjudan-till-larosaten-myndigheter-och-organisationer-att-inkomma-med-synpunkter-till-regeringens-forskningspolitik/   

                                                                             Publishing Info  \
0  Publicerad02 november 2023· Uppdaterad10 november 2023·RemissfrånUtbildningsdepartementet   

                                                                                                                            Collected Links  \
0  /contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

  Outside PDF Text  \
0       NO CONTENT   

                                                                                                                                                Full Collected Links  \
0  https://www.regeringen.se/contentassets/c9981cbd1c054e3fa1a2a917d3e22939/inbjudan-till-larosatena-att-inkomma-med-underlag-till-regeringens-forskningspolitik.pdf   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         